In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#read the data

In [5]:
raw_data <- read_csv("data/HTRU_2.csv", col_names = FALSE)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double()
)



In [7]:
#adding column names manually

In [8]:
raw_data <- rename(raw_data,
       Mean_IP = X1,
       SD_IP = X2,
       ExcessKurtosis_IP = X3,
       Skewness_IP = X4,
       Mean_DS = X5,
       SD_DS = X6,
       ExcessKurtosis_DS = X7,
       Skewness_DS = X8,
       Class = X9)

In [11]:
#removing rows with missing values for the first 8 columns (row count remains the same meaning that there aren't any missing values)

In [15]:
na.omit(raw_data, cols=c("Mean_IP", "SD_IP", "ExcessKurtosis_IP", "Skewness_IP", "Mean_DS", "SD_DS", "ExcessKurtosis_DS", "Skewness_DS"))

Mean_IP,SD_IP,ExcessKurtosis_IP,Skewness_IP,Mean_DS,SD_DS,ExcessKurtosis_DS,Skewness_DS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
140.56250,55.68378,-0.234571412,-0.69964840,3.1998328,19.110426,7.975532,74.24222,0
102.50781,58.88243,0.465318154,-0.51508791,1.6772575,14.860146,10.576487,127.39358,0
103.01562,39.34165,0.323328365,1.05116443,3.1212375,21.744669,7.735822,63.17191,0
136.75000,57.17845,-0.068414638,-0.63623837,3.6429766,20.959280,6.896499,53.59366,0
88.72656,40.67223,0.600866079,1.12349169,1.1789298,11.468720,14.269573,252.56731,0
93.57031,46.69811,0.531904850,0.41672112,1.6362876,14.545074,10.621748,131.39400,0
119.48438,48.76506,0.031460220,-0.11216757,0.9991639,9.279612,19.206230,479.75657,0
130.38281,39.84406,-0.158322759,0.38954045,1.2207358,14.378941,13.539456,198.23646,0
107.25000,52.62708,0.452688025,0.17034738,2.3319398,14.486853,9.001004,107.97251,0


In [ ]:
#create another table where you include the mean next to each variable (copy paste my code and apply it to the training data)

In [40]:
mean_data <- raw_data %>% 
    mutate(AVG_MeanIP = mean(Mean_IP), .after = Mean_IP) %>% 
    mutate(AVG_SDIP = mean(SD_IP), .after = SD_IP) %>%    
    mutate(AVG_ExcessKurtosisIP = mean(ExcessKurtosis_IP), .after = ExcessKurtosis_IP) %>%
    mutate(AVG_SkewnessIP = mean(Skewness_IP), .after = Skewness_IP) %>%
    mutate(AVG_MeanDS = mean(Mean_DS), .after = Mean_DS) %>%
    mutate(AVG_SDDS = mean(SD_DS), .after = SD_DS) %>%
    mutate(AVG_ExcessKurtosisDS = mean(ExcessKurtosis_DS), .after = ExcessKurtosis_DS) %>%
    mutate(AVG_SkewnessDS = mean(Skewness_DS), .after = Skewness_DS)

mean_data

Mean_IP,AVG_MeanIP,SD_IP,AVG_SDIP,ExcessKurtosis_IP,AVG_ExcessKurtosisIP,Skewness_IP,AVG_SkewnessIP,Mean_DS,AVG_MeanDS,SD_DS,AVG_SDDS,ExcessKurtosis_DS,AVG_ExcessKurtosisDS,Skewness_DS,AVG_SkewnessDS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
140.56250,111.08,55.68378,46.54953,-0.234571412,0.4778573,-0.69964840,1.770279,3.1998328,12.6144,19.110426,26.32651,7.975532,8.303556,74.24222,104.8577,0
102.50781,111.08,58.88243,46.54953,0.465318154,0.4778573,-0.51508791,1.770279,1.6772575,12.6144,14.860146,26.32651,10.576487,8.303556,127.39358,104.8577,0
103.01562,111.08,39.34165,46.54953,0.323328365,0.4778573,1.05116443,1.770279,3.1212375,12.6144,21.744669,26.32651,7.735822,8.303556,63.17191,104.8577,0
136.75000,111.08,57.17845,46.54953,-0.068414638,0.4778573,-0.63623837,1.770279,3.6429766,12.6144,20.959280,26.32651,6.896499,8.303556,53.59366,104.8577,0
88.72656,111.08,40.67223,46.54953,0.600866079,0.4778573,1.12349169,1.770279,1.1789298,12.6144,11.468720,26.32651,14.269573,8.303556,252.56731,104.8577,0
93.57031,111.08,46.69811,46.54953,0.531904850,0.4778573,0.41672112,1.770279,1.6362876,12.6144,14.545074,26.32651,10.621748,8.303556,131.39400,104.8577,0
119.48438,111.08,48.76506,46.54953,0.031460220,0.4778573,-0.11216757,1.770279,0.9991639,12.6144,9.279612,26.32651,19.206230,8.303556,479.75657,104.8577,0
130.38281,111.08,39.84406,46.54953,-0.158322759,0.4778573,0.38954045,1.770279,1.2207358,12.6144,14.378941,26.32651,13.539456,8.303556,198.23646,104.8577,0
107.25000,111.08,52.62708,46.54953,0.452688025,0.4778573,0.17034738,1.770279,2.3319398,12.6144,14.486853,26.32651,9.001004,8.303556,107.97251,104.8577,0


In [ ]:
# num_obs <- nrow(raw_data)
# raw_data %>% 
#    group_by(Class) %>% 
#    summarize(
#        count = n(),
#        percentage = n() / num_obs * 100 
#   )

In [ ]:
#i can add visualizations after we create the training data!